### Code to time models against sample feeder images

In [1]:
import tensorflow as tf
from tensorflow import keras
from PIL import Image
import numpy as np
import os
import time


2025-04-13 19:05:41.147245: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-13 19:05:42.126895: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/lib/x86_64-linux-gnu/:/opt/conda/lib
2025-04-13 19:05:42.126993: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local

In [2]:
image_dir = 'home/pi/batch_test'  # relative to birdclassifier dir in jupyter nb

In [3]:
# get the classes, number space text with a line end 
classes_dict = {}
with open('NAbirdsClasses.txt', 'r') as f:
    for line in f:
        line = line.strip()
        if line:  # Ignore empty lines
            parts = line.split(' ', 1)
            if len(parts) == 2 and parts[0].isdigit():
                key = int(parts[0])
                value = parts[1]
                classes_dict[key] = value
# print(classes_dict)

In [4]:
def list_models(prefix='Experiment5'):
    matching_files = []
    for filename in os.listdir():  # current dir
        if filename.startswith(prefix) and filename.endswith('.h5'):
            matching_files.append(filename)
    return matching_files

In [5]:
def load_model(model_path):
    # load the model and get input size
    model = keras.models.load_model(model_path)
    # print(model.input_shape)  
    input_shape = model.input_shape[1:3]  # height, width, channel in format (None, 224, 224, 3)
    return model, input_shape

In [6]:
def preprocess_image(image_path, target_size):
    img = Image.open(image_path).resize(target_size)
    img_array = np.array(img)
    # img_array = img_array / 255.0  # done in model? 
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension, we are doing this one at a time
    return img_array

In [7]:
def batch_predict_model(image_files, image_dir, model, input_shape, classes_dict):
    # includes load time, but that needs to happen for resizing
    start_time = time.time()
    images_count = len(image_files)
    for ii, image_file in enumerate(image_files):
        image_path = os.path.join(image_dir, image_file)
        try:
            preprocessed_image = preprocess_image(image_path, input_shape)
            prediction = model.predict(preprocessed_image)
            predicted_class_index = np.argmax(prediction)
            confidence = prediction[0][predicted_class_index] * 100
            print(f'image {ii} of {images_count}: {image_file}, Prediction: {predicted_class_index}:{classes_dict[predicted_class_index]}, Confidence: {confidence:.2f}%')

        except Exception as e:
            print(f'error processing image {image_file}: {e}')

    stop_time = time.time()
    total_run_time = stop_time - start_time
    print(f'\n run time: {total_run_time:.2f} seconds')
    return

In [9]:
model_files = list_models()
image_files = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]

for model_filename in model_files:
    print('*************************')
    print(f'model: {model_filename}')
    try:
        model, input_shape = load_model(model_filename)
        batch_predict_model(image_files, image_dir, model, input_shape, classes_dict)
        print('')
    except Exception as e:
        print (f'error with model {model_filename} as {e}')
    

*************************
model: Experiment5:MobileNetV2MobileNetV2.h5
1/1 [==============================] - 1s 673ms/step
image 0 of 160: raw_2025-02-18-14-18-381639(NorthernCardinal [(166, 216, 475, 511)]).jpg, Prediction: 19:Northern Cardinal Male, Confidence: 98.92%
1/1 [==============================] - 0s 24ms/step
image 1 of 160: raw_2025-02-19-15-51-162397(NorthernCardinal [(38, 224, 445, 573)]).jpg, Prediction: 5:Black-Capped Chickadee, Confidence: 59.59%
1/1 [==============================] - 0s 23ms/step
image 2 of 160: raw_2025-01-23-13-22-521138(MourningDove [(4, 205, 430, 523)]).jpg, Prediction: 2:American Robin, Confidence: 18.14%
1/1 [==============================] - 0s 23ms/step
image 3 of 160: raw_2025-02-12-10-42-311361(PurpleFinch [(3, 310, 303, 610)]).jpg, Prediction: 17:Purple Finch Male, Confidence: 8.85%
1/1 [==============================] - 0s 23ms/step
image 4 of 160: raw_2025-02-12-10-42-411367(PurpleFinch [(1, 291, 326, 617)]).jpg, Prediction: 9:Common Gr

2025-04-13 19:07:28.140968: W tensorflow/tsl/framework/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.75MiB (rounded to 3932160)requested by op Fill
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2025-04-13 19:07:28.141046: I tensorflow/tsl/framework/bfc_allocator.cc:1034] BFCAllocator dump for GPU_0_bfc
2025-04-13 19:07:28.141058: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (256): 	Total Chunks: 434, Chunks in use: 434. 108.5KiB allocated for chunks. 108.5KiB in use in bin. 43.1KiB client-requested in use in bin.
2025-04-13 19:07:28.141065: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (512): 	Total Chunks: 466, Chunks in use: 466. 299.8KiB allocated for chunks. 299.8KiB in use in bin. 250.2KiB client-requested in use in bin.
2025-04-13 19:07:28.141072: I tensorflow/tsl/frame